In [1]:
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import glob
import random

In [3]:
data_dir = './dataset/多输出训练模型数据集'

In [4]:
data_root = pathlib.Path(data_dir)

In [5]:
all_image_path = list(data_root.glob('*/*'))

In [6]:
all_image_path = [str(path) for path in all_image_path]
all_image_path[0:5]

['dataset\\多输出训练模型数据集\\black_jeans\\00000000.jpg',
 'dataset\\多输出训练模型数据集\\black_jeans\\00000001.jpeg',
 'dataset\\多输出训练模型数据集\\black_jeans\\00000002.jpeg',
 'dataset\\多输出训练模型数据集\\black_jeans\\00000003.jpg',
 'dataset\\多输出训练模型数据集\\black_jeans\\00000004.jpg']

In [7]:
random.shuffle(all_image_path)
all_image_path[0:5]

['dataset\\多输出训练模型数据集\\red_dress\\00000333.jpg',
 'dataset\\多输出训练模型数据集\\blue_jeans\\00000086.jpg',
 'dataset\\多输出训练模型数据集\\red_dress\\00000114.jpg',
 'dataset\\多输出训练模型数据集\\black_jeans\\00000351.jpg',
 'dataset\\多输出训练模型数据集\\blue_shirt\\00000011.jpg']

In [8]:
label_name = sorted(item.name for item in data_root.glob('*/'))
label_name

['black_jeans',
 'black_shoes',
 'blue_dress',
 'blue_jeans',
 'blue_shirt',
 'red_dress',
 'red_shirt']

In [9]:
color_label_names = set(name.split('_')[0] for name in label_name)
color_label_names

{'black', 'blue', 'red'}

In [10]:
item_label_names = set(name.split('_')[1] for name in label_name)
item_label_names

{'dress', 'jeans', 'shirt', 'shoes'}

In [11]:
color_label_to_index = dict((name, index) for index, name in enumerate(color_label_names))
color_label_to_index

{'red': 0, 'black': 1, 'blue': 2}

In [12]:
item_label_to_index = dict((name, index) for index, name in enumerate(item_label_names))
item_label_to_index

{'dress': 0, 'shoes': 1, 'jeans': 2, 'shirt': 3}

In [13]:
all_image_labels = [pathlib.Path(path).parent.name for path in all_image_path]
all_image_labels[0:5]

['red_dress', 'blue_jeans', 'red_dress', 'black_jeans', 'blue_shirt']

In [14]:
color_label = [color_label_to_index[label.split('_')[0]] for label in all_image_labels]
color_label[0:5]

[0, 2, 0, 1, 2]

In [15]:
item_label = [item_label_to_index[label.split('_')[1]] for label in all_image_labels]
item_label[0:5]

[0, 2, 0, 2, 3]

In [16]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224,224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    return image

In [17]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_path)
path_ds

<TensorSliceDataset shapes: (), types: tf.string>

In [18]:
image_ds = path_ds.map(load_and_preprocess_image)
image_ds

<MapDataset shapes: (224, 224, 3), types: tf.float32>

In [19]:
label_ds = tf.data.Dataset.from_tensor_slices((color_label, item_label))

In [20]:
for e in label_ds.take(3):
    print(e[0].numpy(), e[1].numpy())

0 0
2 2
0 0


In [21]:
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
image_label_ds

<ZipDataset shapes: ((224, 224, 3), ((), ())), types: (tf.float32, (tf.int32, tf.int32))>

In [22]:
image_count = len(all_image_path)
test_count = int(image_count*0.2)
train_count = image_count-test_count

train_data = image_label_ds.skip(test_count)
test_data = image_label_ds.take(test_count)

train_count

2020

In [23]:
BATCH_SIZE = 16

In [24]:
train_data = train_data.shuffle(buffer_size=train_count).repeat(-1)
train_data =train_data.batch(BATCH_SIZE)
test_data = test_data.batch(BATCH_SIZE)
train_data

<BatchDataset shapes: ((None, 224, 224, 3), ((None,), (None,))), types: (tf.float32, (tf.int32, tf.int32))>

In [25]:
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top = False,
                                               weights='imagenet') #加载预训练权重

In [26]:
mobile_net.trianable = False #设置不可训练

In [27]:
inputs = tf.keras.Input(shape=(224, 224, 3))

In [28]:
x = mobile_net(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x1 = tf.keras.layers.Dense(1024, activation='relu')(x)
out_color = tf.keras.layers.Dense(len(color_label_names), 
                                  activation='softmax',
                                  name='out_color')(x1)
x2 = tf.keras.layers.Dense(1024, activation='relu')(x)
out_item = tf.keras.layers.Dense(len(item_label_names), 
                                 activation='softmax',
                                 name='out_item')(x2)

In [29]:
model = tf.keras.Model(inputs = inputs,
                       outputs = [out_color, out_item])

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
mobilenetv2_1.00_224 (Model)    (None, 7, 7, 1280)   2257984     input_2[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 1280)         0           mobilenetv2_1.00_224[1][0]       
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         1311744     global_average_pooling2d[0][0]   
______________________________________________________________________________________________

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss={'out_color':'sparse_categorical_crossentropy',
                    'out_item':'sparse_categorical_crossentropy',},
              metrics=['acc']
             )

In [32]:
train_steps = train_count//BATCH_SIZE
test_steps = test_count//BATCH_SIZE
test_steps

31

In [35]:
history = model.fit(train_data,
                   epochs=5,
                   steps_per_epoch=train_steps,
                   validation_data=test_data,
                   validation_steps=test_steps
         ) 

Train for 126 steps, validate for 31 steps
Epoch 1/5
126/126 [==============================] - 72s 568ms/step - loss: 0.0542 - out_color_loss: 0.0311 - out_item_loss: 0.0232 - out_color_acc: 0.9891 - out_item_acc: 0.9936 - val_loss: 0.1572 - val_out_color_loss: 0.0858 - val_out_item_loss: 0.0714 - val_out_color_acc: 0.9758 - val_out_item_acc: 0.9758
Epoch 2/5
126/126 [==============================] - 69s 546ms/step - loss: 0.0612 - out_color_loss: 0.0278 - out_item_loss: 0.0334 - out_color_acc: 0.9901 - out_item_acc: 0.9921 - val_loss: 0.1087 - val_out_color_loss: 0.0503 - val_out_item_loss: 0.0584 - val_out_color_acc: 0.9819 - val_out_item_acc: 0.9819
Epoch 3/5
126/126 [==============================] - 70s 555ms/step - loss: 0.0394 - out_color_loss: 0.0256 - out_item_loss: 0.0137 - out_color_acc: 0.9926 - out_item_acc: 0.9945 - val_loss: 0.1422 - val_out_color_loss: 0.0714 - val_out_item_loss: 0.0708 - val_out_color_acc: 0.9798 - val_out_item_acc: 0.9819
Epoch 4/5
126/126 [========

In [36]:
model.evaluate( test_data,verbose=0,steps=1)

[0.009910398162901402, 0.004384175, 0.005526223, 1.0, 1.0]